# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2020-07-13	Amortized Population Gibbs Samplers with Neural Sufficient Statistics	37th International Conference on Machine Learning	We develop amortized population Gibbs (APG) samplers, a class of scalable methods that frame structured variational inference as adaptive importance sampling. APG samplers construct high-dimensional proposals by iterating over updates to lower-dimensional blocks of variables. We train each conditional proposal by minimizing the inclusive KL divergence with respect to the conditional posterior. To appropriately account for the size of the input data, we develop a new parameterization in terms of neural sufficient statistics. Experiments show that APG samplers can be used to train highly-structured deep generative models in an unsupervised manner, and achieve substantial improvements in inference accuracy relative to standard autoencoding variational methods. 	"@InProceedings{pmlr-v119-wu20h,
  title = 

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2020-07-13,Amortized Population Gibbs Samplers with Neura...,37th International Conference on Machine Learning,We develop amortized population Gibbs (APG) sa...,"@InProceedings{pmlr-v119-wu20h,\n title = \t ...",Apg-icml-2020,http://esennesh.github.io/files/wu20h.pdf
1,2020-11-14,Learning a Deep Generative Model like a Progra...,AAAI Fall 2020 Symposium on Conceptual Abstrac...,Humans surpass the cognitive abilities of most...,"@article{sennesh2020learning,\n title={Learni...",freecat-aaai-2020-symposium,http://esennesh.github.io/files/freecat_aaai_s...
2,2020-12-06,Neural Topographic Factor Analysis for fMRI Data,Advances in Neural Information Processing Syst...,Neuroimaging studies produce gigabytes of spat...,"@inproceedings{NEURIPS2020_8c3c27ac,\n author ...",Ntfa-neurips-2020,http://esennesh.github.io/files/ntfa_neurips_2...
3,2021-07-27,Learning proposals for probabilistic programs ...,Proceedings of the Thirty-Seventh Conference o...,We develop operators for construction of prop...,"@InProceedings{pmlr-v161-stites21a,\n title =...",Combinators-uai-2021,http://esennesh.github.io/files/stites21a.pdf
4,2021-12-20,"Interoception as modeling, allostasis as control",Biological Psychology,The brain regulates the body by anticipating i...,"@article{sennesh2022interoception,\n title={I...",Allostasis-biopsych-2021,http://esennesh.github.io/files/allostasis_bio...
5,2022-03-29,A Computational Neural Model for Mapping Degen...,Neuroinformatics,Degeneracy in biological systems refers to a m...,"@article{khan2022computational,\n title={A co...",Ntfa-neuroinformatics-2022,http://esennesh.github.io/files/ntfa_neuroinfo...
6,2022-05-09,A Probabilistic Generative Model of Free Categ...,arxiv,Applied category theory has recently developed...,"@article{sennesh2022probabilistic,\n title={A...",Freecat-arxiv-2022,http://esennesh.github.io/files/freecat_arxiv_...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2016-04-08-structured-gotos-slightly-harmful.md
2018-05-02-creer-de-tete-de-nombreux-mots-de-passe-inviolables-et-inoubliables.md
2018-10-04-combinators-modeling-inference.md
2018-10-04-Combinators-probprog-2018.md
2018-12-07-bnp-neurips-combinators.md
2019-06-24-arxiv-neuraltfa.md
2020-07-13-Apg-icml-2020.md
2020-11-14-freecat-aaai-2020-symposium.md
2020-12-06-Ntfa-neurips-2020.md
2021-07-27-Combinators-uai-2021.md
2021-12-20-Allostasis-biopsych-2021.md
2022-03-29-Ntfa-neuroinformatics-2022.md
2022-05-09-Freecat-arxiv-2022.md


In [7]:
!cat ../_publications/2021-07-27-Combinators-uai-2021.md

---
title: "Learning proposals for probabilistic programs with inference combinators"
collection: publications
permalink: /publication/2021-07-27-Combinators-uai-2021
excerpt: ' We develop operators for construction of proposals in probabilistic programs, which we refer to as inference combinators. Inference combinators define a grammar over importance samplers that compose primitive operations such as application of a transition kernel and importance resampling. Proposals in these samplers can be parameterized using neural networks, which in turn can be trained by optimizing variational objectives. The result is a framework for user-programmable variational methods that are correct by construction and can be tailored to specific models. We demonstrate the flexibility of this framework by implementing advanced variational methods based on amortized Gibbs sampling and annealing. '
date: 2021-07-27
venue: 'Proceedings of the Thirty-Seventh Conference on Uncertainty in Artificial Intellig